### Setup libraries

In [5]:
# pip install annoy # conda install -c conda-forge python-annoy
# pip install compress_fasttext
# pip install python-telegram-bot --upgrade
# pip install pymorphy2
# pip install stop_words

### Import libraries

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:72.5% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd

import io
import re
import time
import unicodedata
from pathlib import Path
from linecache import getline
from tqdm.notebook import tqdm
from collections import Counter
import functools

import annoy
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import compress_fasttext
from gensim.models import FastText, KeyedVectors
from gensim.models.fasttext import FastTextKeyedVectors


from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

import matplotlib.pyplot as plt

In [3]:
plt.rcParams.update({'font.size': 14})
pd.set_option('precision', 3)
pd.set_option('max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 300)

In [4]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#     raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

### Paths to directories and files

In [5]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [6]:
DRAFT_DATASET_PATH = 'E:/kaggle/mailru/Otvety.txt'
# DRAFT_DATASET_PATH = '/content/gdrive/MyDrive/Colab Notebooks/my_projects/Otvety.txt'

### Loading data

In [7]:
lines = io.open(DRAFT_DATASET_PATH, encoding='UTF-8').read()

### Preprocessing data

In [8]:
def preprocess_text(w):
    w = re.sub(':\)', '', w)
    w = re.sub('[)"]', '', w)
    w = re.sub('<[^>]+>', ' ', w)
    
    w = re.sub('\s*\?\s*\.', '?', w)
    w = re.sub('\s*\!\s*\.', '!', w)
    w = re.sub('\s*\.', '.', w)
    w = re.sub('\.+', '.', w)
    
    w = re.sub('---', 'QUESTION', w)
    
    return w

In [9]:
# print(type(lines))
print(len(lines))

1072593541


In [10]:
NUM_EXAMPLES = 7777777
text = preprocess_text(str(lines[:NUM_EXAMPLES]))
print(text[:1000])


QUESTION
вопрос о ТДВ давно и хорошо отдыхаем ЛИЧНО ВАМ здесь кого советовали завести? 
хомячка. 
мужика, йопаря, собачку и 50 кошек. 
Общение! 
паучка. 
Да пол мне бы памыть! А таг то ни чо. Типа ни каво! 
я тут вообще что бы пообщаться. 
А мне советовали сиси завести. 
Ну, слава богу, мужика завести ещё не советовали А вот сватать к кому только не сватали. 
мне тут советовали завести любовника, мужа и много кошек  приветик. 
QUESTION
Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые. 
меня вобще прикалывает эта тема. 
когда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы. 
QUESTION
Что делать, сегодня нашёл 2 миллиона рублей? 
Если это счастье  действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни

In [11]:
print(len(text))

7540254


In [12]:
morpher = MorphAnalyzer()
sw = set(get_stop_words('ru'))

In [13]:
def split_by_sentence(sent):
    sent = text.split('\nQUESTION\n')[1:]
    
    questions = []
    contexts = []

    for se in sent:
        se = se.split('\n')
#         se = [i for i in se if (i not in string.punctuation)]
        se = [morpher.parse(i.lower())[0].normal_form for i in se]
#         se = [i for i in se if i not in sw and i != '']

        questions.append(se[0].strip())
        contexts.append(' '.join([f' {s}' for s in se[1:]]))
    
    return questions, contexts

In [14]:
questions, contexts = split_by_sentence(text)
questions[0], contexts[0]

('вопрос о тдв давно и хорошо отдыхаем лично вам здесь кого советовали завести?',
 ' хомячка.   мужика, йопаря, собачку и 50 кошек.   общение!   паучка.   да пол мне бы памыть! а таг то ни чо. типа ни каво!   я тут вообще что бы пообщаться.   а мне советовали сиси завести.   ну, слава богу, мужика завести ещё не советовали а вот сватать к кому только не сватали.   мне тут советовали завести любовника, мужа и много кошек  приветик. ')

In [15]:
print(len(questions[-1])), print(len(contexts[-1]))

102
2694


(None, None)

In [16]:
df = pd.DataFrame({'questions':questions, 'contexts':contexts})

In [17]:
df.isnull().sum()

questions    0
contexts     0
dtype: int64

In [18]:
df.head()

,questions,contexts
0,вопрос о тдв давно и хорошо отдыхаем лично вам здесь кого советовали завести?,"хомячка. мужика, йопаря, собачку и 50 кошек. общение! паучка. да пол мне бы памыть! а таг то ни чо. типа ни каво! я тут вообще что бы пообщаться. а мне советовали сиси завести. ну, слава богу, мужика завести ещё не советовали а вот сватать к кому только не сватали. мне тут совет..."
1,"как парни относятся к цветным линзам? если у девушки то зеленые глаза, то голубые.","меня вобще прикалывает эта тема. когда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы."
2,"что делать, сегодня нашёл 2 миллиона рублей?","если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке. такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. можно нарваться на бабушку конечно, которая хотела помоч внуку с покупк..."
3,эбу в двенашке называется итэлма что за эбу?,"эбу — электронный блок управления двигателем автомобиля, его другое название — контроллер. он принимает информацию от многочисленных датчиков, обрабатывает ее по особым алгоритмам и, отталкиваясь от полученных данных, отдает команды исполнительным устройствам системы. это завод. а эбу может и..."
4,академия вампиров. сколько на даный момент частей книги академия вампиров?,"4. охотники и жертвы, ледяной укус, поцелуй тьмы, кровная клятва. на данное время их 6. часть 5- оковы для призрака (духовная связь часть 6-последняя жертва."


### ml_baseline

In [19]:
def normalize_answer(text):
    """Lower text and remove punctuation and extra whitespace."""
    return ' '.join(re.findall(r"\w+", text)).lower()

In [20]:
def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = len(common)
    if num_same == 0:
        return 0.0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [21]:
def sentence_to_word(sentences):
    sentences_in_words = list()
    for sentence in sentences:
        sentences_in_words.append(tuple(normalize_answer(sentence).split()))
    return sentences_in_words

In [22]:
def text_to_sentence(text):
    sentences = text.split(".")
    return [s.strip() for s in sentences if s.strip() != '']

In [23]:
def uniq_words(text):
    return set(re.findall("\w+", text))

In [30]:
def calculate_idfs(data):
    counter_context = Counter()
    uniq_contexts = data['contexts'].unique()
    for contexts in tqdm(uniq_contexts, desc="calc idf"):
        set_words = uniq_words(contexts)
        counter_context.update(set_words)

    num_docs = uniq_contexts.shape[0]
    idfs = {}
    for word in counter_context:
        idfs[word] = np.log(num_docs / counter_context[word])
    return idfs

In [45]:
class FeatureMaker(object):

    """
    Класс для построения признаков из текста
    Если есть колонка answer, то добавляется колонка target, если нет, то не добавляется.
    """

    def __init__(self, dataframe, idfs):
        self.data = dataframe.sort_values('contexts')
        self.idfs = idfs

        self.result = {}
        self.keys = {}
        self.rev_keys = {}
        self.max_key = 0

        self.morph = pymorphy2.MorphAnalyzer()
        # Максимальная длинна в словах кандидатов
        self.MAX_SPAN_LEN = 5
        self.MORPH_TAGS = {
            'NOUN', 'VERB', 'ADJF', 'ADJS', 'NUMR', 'PREP', 'CONJ', 'PRCL',
            'INTJ', 'GRND', 'COMP', 'INFN', 'PRTF', 'PRTS', 'ADVB', 'NPRO',
            'PRED', 'LATN', 'ROMN', 'UNKN'
        }
        columns = ['ADJF', 'ADJS', 'ADVB', 'COMP', 'CONJ', 'GRND', 'IDF (Span)', 'INFN',
                   'INTJ', 'LATN', 'Left Length', 'Match IDF  (Span)',
                   'Match IDF (Right of Span)', 'Match IDF (Whole Sentence)',
                   'Match TF IDF (Left of Span)', 'NOUN', 'NPRO', 'NUMR', 'PRCL', 'PRED',
                   'PREP', 'PRTF', 'PRTS', 'ROMN', 'Right Length', 'Sentence Length',
                   'Span Length', 'UNKN', 'VERB']
        if 'answer' in self.data.columns:
            columns += ['target']
        self.columns = columns
        # preallocate data
        for key in columns:
            self.result[key] = [None] * self.data.shape[0] * 100

    def make(self):
        for data_ind in tqdm(self.data.index.values, desc="calc features"):
            self.generate_candidates_with_features(self.data.loc[data_ind])
        for col_name in self.result:
            self.result[col_name] = self.result[col_name][:self.max_key]
        return pd.DataFrame.from_dict(self.result).fillna(0, inplace=False)

    def generate_candidates_with_features(self, data_row):
        contexts = data_row["contexts"]
        questions = data_row["questions"]
        if 'answer' in data_row:
            answer = data_row["answer"]
        else:
            answer = None
        sentences = text_to_sentence(contexts)
        sentences_in_words = sentence_to_word(sentences)
        question_in_words = sentence_to_word([questions])[0]

        # определение предложения для поиска ответа
        # выбираем такое предложение, которое сильнее всего пересекается с вопросом
        # это эвристика для уменьшения числа кандидатов
        sentence_id = self.get_max_match_sentance_id(sentences_in_words,
                                                     question_in_words)
        sentence = sentences_in_words[sentence_id]
        sentenceMorphy = self.convert_morphy(sentence)

        # генерируем все воможные кандидаты для выбранного предложения и строим для них признаки
        for i in range(len(sentence)):
            for j in range(i + 1, min(i + 1 + self.MAX_SPAN_LEN, len(sentence) + 1)):
                span_hash = self.calculate_key(sentence_id, (i, j))
                self.generate_features_for_span(
                    span_hash, sentence, question_in_words, (i, j),
                    sentenceMorphy, answer)

        # добавляем еще одного кандидата - все предложение
        span = (0, len(sentence))
        span_hash = self.calculate_key(sentence_id, span)
        self.generate_features_for_span(span_hash, sentence, question_in_words,
                                        span, sentenceMorphy, answer)

    def generate_features_for_span(self, key, sentence, questions, span,
                                   sentenceMorphy, answer):
        """
        Метод для генерации признаков для ответа-кандидата
        """
        self.add_column(key, 'Left Length', span[0])
        self.add_column(key, 'Right Length', len(sentence) - span[1])
        self.add_column(key, 'Sentence Length', len(sentence))
        self.add_column(key, 'Span Length', span[1] - span[0])

        self.add_idf_feature(key, 'IDF (Span)', sentence, span)

        self.add_idf_match_feature(key, 'Match TF IDF (Left of Span)',
                                   sentence, questions, (0, span[0]))
        self.add_idf_match_feature(key, 'Match IDF (Right of Span)', sentence,
                                   questions, (span[1], len(sentence)))
        self.add_idf_match_feature(key, 'Match IDF (Whole Sentence)', sentence,
                                   questions, (0, len(sentence)))
        self.add_idf_match_feature(key, 'Match IDF  (Span)', sentence,
                                   questions, span)

        # добавляем признаки на основе частей речи слов
        self.add_morph_tag_feature(key, sentenceMorphy, span)

        if answer is not None:
            self.add_column(key, 'target', f1_score(' '.join(sentence[span[0]:span[1]]), answer))

    def get_max_match_sentance_id(self, sentences_in_words, question_in_words):
        """
        поиск предложения в параграфе, которое сильнее всего пересекается с вопросом
        """
        max_overlap = -1
        max_match_sentance_id = None

        question_words = set(question_in_words)
        for sentance_id in range(len(sentences_in_words)):
            overlap = len(set(sentences_in_words[sentance_id]) & question_words)
            if overlap > max_overlap:
                max_overlap = overlap
                max_match_sentance_id = sentance_id

        return max_match_sentance_id

    def add_column(self, key, name, value):
        """
        метод для добавления колонки в результирующие данные
        """
        if len(self.result[name]) <= key:
            self.result[name] += [None] * (key + 1 - len(self.result[name]))
        self.result[name][key] = value

    def add_idf_feature(self, key, name, sentence, span):
        """
        добавление признаков вида \sum_{w in span} idf[w]
        """
        self.add_column(key, name, self.calculate_sum_idf_sentence(sentence, span))

    @functools.lru_cache(maxsize=2 ** 14)
    def calculate_sum_idf_sentence(self, sentence, span):
        sum_idf = 0.0
        for w in sentence[span[0]:span[1]]:
            if w in self.idfs:
                sum_idf += self.idfs[w]
        return sum_idf

    @functools.lru_cache(maxsize=2 ** 14)
    def calculate_sum_idf_sentence_question(self, sentence, questions, span):
        sum_idf = 0.0
        for w in sentence[span[0]:span[1]]:
            if w in questions and w in self.idfs:
                sum_idf += self.idfs[w]
        return sum_idf

    def add_idf_match_feature(self, key, name, sentence, questions, span):
        """
        добавление признаков вида \sum_{w in span and w in question} idf[w]
        """
        self.add_column(key, name,
                        self.calculate_sum_idf_sentence_question(sentence, questions, span))

    def calculate_key(self, sentence_id, span):
        """
        метод для генерации уникального ключа в результирующей таблице
        """
        key_tuple = (sentence_id, span[0], span[1])
        if key_tuple not in self.keys:
            self.keys[key_tuple] = self.max_key
            self.rev_keys[self.max_key] = key_tuple
            self.max_key += 1
        return self.keys[key_tuple]

    @functools.lru_cache(maxsize=2 ** 17)
    def morphize(self, word):
        return self.morph.parse(word)[0].tag.grammemes & self.MORPH_TAGS

    @functools.lru_cache(maxsize=2 ** 4)
    def convert_morphy(self, sentence):
        return tuple((self.morphize(word) for word in sentence))

    @functools.lru_cache(maxsize=2 ** 8)
    def get_morph_counter(self, sentenceMorphy, span):
        count_grammes = Counter((grammem for grammems in sentenceMorphy[span[0]:span[1]]
                                 for grammem in grammems))
        return count_grammes

    def add_morph_tag_feature(self, key, sentenceMorphy, span):
        count_grammes = self.get_morph_counter(sentenceMorphy, span)
        for k in self.MORPH_TAGS:
            self.add_column(key, k, count_grammes.get(k, 0))

    def get_span(self, key):
        """
        reverse opeartion for found selected answer
        """
        sentence_id, i, j = self.rev_keys[key]
#         contexts = self.data[(self.data["question_id"] == question_id) & (self.data['paragraph_id'] == paragraph_id)].iloc[0]["paragraph"]

        sentences = text_to_sentence(contexts)
        sentences_in_words = sentence_to_word(sentences)
        return ' '.join(sentences_in_words[sentence_id][i:j])

In [39]:
idfs = calculate_idfs(df)

In [43]:
import pymorphy2

In [46]:
maker = FeatureMaker(df.head(10000), idfs)
train = maker.make()

TypeError: list indices must be integers or slices, not NoneType

In [58]:
# question_in_words

In [59]:
# sentences_in_words

In [61]:
df['predictions'] = None
for data_ind in df.index.values:
    full_sentance = get_max_match_sentance(df.loc[data_ind])
    df.loc[data_ind,('predictions')] = full_sentance

In [83]:
df.head()

,questions,contexts,predictions
0,вопрос о тдв давно и хорошо отдыхаем лично вам здесь кого советовали завести?,"хомячка. мужика, йопаря, собачку и 50 кошек. общение! паучка. да пол мне бы памыть! а таг то ни чо. типа ни каво! я тут вообще что бы пообщаться. а мне советовали сиси завести. ну, слава богу, мужика завести ещё не советовали а вот сватать к кому только не сватали. мне тут совет...",хомячка
1,"как парни относятся к цветным линзам? если у девушки то зеленые глаза, то голубые.","меня вобще прикалывает эта тема. когда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы.",меня вобще прикалывает эта тема
2,"что делать, сегодня нашёл 2 миллиона рублей?","если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке. такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. можно нарваться на бабушку конечно, которая хотела помоч внуку с покупк...","если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке"
3,эбу в двенашке называется итэлма что за эбу?,"эбу — электронный блок управления двигателем автомобиля, его другое название — контроллер. он принимает информацию от многочисленных датчиков, обрабатывает ее по особым алгоритмам и, отталкиваясь от полученных данных, отдает команды исполнительным устройствам системы. это завод. а эбу может и...","эбу — электронный блок управления двигателем автомобиля, его другое название — контроллер"
4,академия вампиров. сколько на даный момент частей книги академия вампиров?,"4. охотники и жертвы, ледяной укус, поцелуй тьмы, кровная клятва. на данное время их 6. часть 5- оковы для призрака (духовная связь часть 6-последняя жертва.",4


In [33]:
def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [34]:
def get_score(df_solution, df_predictions):
    score = {
        'f1':
        np.mean([
            f1_score(prediction, answer) for answer, prediction in zip(df_solution, df_predictions)
        ]),
    }
    return score

In [35]:
# from collections import Counter

In [114]:
# df['answer'] = df['predictions']

In [115]:
# del df['answer']

In [116]:
# if 'answer' in df.columns.values:
#     print(get_score(df['predictions'].values, df['answer'].values))

In [118]:
# df.columns.values

In [119]:
df.head()

,questions,contexts,predictions,answer
0,вопрос о тдв давно и хорошо отдыхаем лично вам здесь кого советовали завести?,"хомячка. мужика, йопаря, собачку и 50 кошек. общение! паучка. да пол мне бы памыть! а таг то ни чо. типа ни каво! я тут вообще что бы пообщаться. а мне советовали сиси завести. ну, слава богу, мужика завести ещё не советовали а вот сватать к кому только не сватали. мне тут совет...",хомячка,хомячка
1,"как парни относятся к цветным линзам? если у девушки то зеленые глаза, то голубые.","меня вобще прикалывает эта тема. когда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы.",меня вобще прикалывает эта тема,меня вобще прикалывает эта тема
2,"что делать, сегодня нашёл 2 миллиона рублей?","если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке. такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. можно нарваться на бабушку конечно, которая хотела помоч внуку с покупк...","если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке","если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке"
3,эбу в двенашке называется итэлма что за эбу?,"эбу — электронный блок управления двигателем автомобиля, его другое название — контроллер. он принимает информацию от многочисленных датчиков, обрабатывает ее по особым алгоритмам и, отталкиваясь от полученных данных, отдает команды исполнительным устройствам системы. это завод. а эбу может и...","эбу — электронный блок управления двигателем автомобиля, его другое название — контроллер","эбу — электронный блок управления двигателем автомобиля, его другое название — контроллер"
4,академия вампиров. сколько на даный момент частей книги академия вампиров?,"4. охотники и жертвы, ледяной укус, поцелуй тьмы, кровная клятва. на данное время их 6. часть 5- оковы для призрака (духовная связь часть 6-последняя жертва.",4,4


In [ ]:
def evaluate(inp_sentence):
    start_token = [tokenizer_en.vocab_size]
    end_token = [tokenizer_en.vocab_size + 1]
    
    inp_sentence = start_token + tokenizer_en.encode(inp_sentence) + end_token
    encoder_input = tf.expand_dims(inp_sentence, 0)

    decoder_input = [tokenizer_en.vocab_size]
    output = tf.expand_dims(decoder_input, 0)

    for i in range(MAX_LEN):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input, output, False,
            enc_padding_mask, combined_mask, dec_padding_mask
        )
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == tokenizer_en.vocab_size+1:
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights

In [ ]:
def text_generator(sentence):
    result, attention_weights = evaluate(sentence)
    predicted_sentence = tokenizer_en.decode([i for i in result if i < tokenizer_en.vocab_size - 1])
    
    print(f'Input: {sentence}')
    # print(f'Predicted message: {predicted_sentence}')
    return predicted_sentence

### Evaluate

In [10]:
# text_generator('Что такое ЭБУ?')

In [11]:
# text_generator('Чем заняться?')

In [12]:
# text_generator('Что приготовить?')

In [ ]:
# text_generator('Что посмотреть?')

In [ ]:
# text_generator('Что послушать?')

In [ ]:
# text_generator('Что почитать?')

In [ ]:
# text_generator('Как подключить интернет?')

In [13]:
# text_generator('Как починить машину?')

### python-telegram-bot

In [114]:
updater = Updater(token='')
dispatcher = updater.dispatcher

In [117]:
def answer(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text=text_generator(update.message.text))
    
answer_handler = MessageHandler(Filters.text, answer)
dispatcher.add_handler(answer_handler)

In [118]:
chat = True

if chat:
    updater.start_polling()
    print('@MyPersonalAssistanBot - online')
else:
    updater.stop()
    print('@MyPersonalAssistanBot - offline')

@MyPersonalAssistanBot - online
